## Statisitically Testing 2P Responses to Stimulus

### Environment Setup

In [1]:
### if running on Google Colab, run this cell once, then restart the runtime and run the rest of the notebook
import os
if "COLAB_GPU" in os.environ:
    !wget https://raw.githubusercontent.com/AllenInstitute/openscope_databook/main/requirements.txt
    %pip install -r requirements.txt --user

In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

from dandi import dandiapi
from pynwb import NWBHDF5IO
from scipy import interpolate

%matplotlib inline

### Downloading Ophys File
Change the values below to download the file you’re interested in. Set `dandiset_id` and `dandi_filepath` to correspond to the dandiset id and filepath of the file you want. If you’re accessing an embargoed dataset, change `authenticate` to True, and set `dandi_api_key` to your DANDI API key. If you want to stream a file instead of downloading it, checkout Streaming an NWB File with fsspec.

In [3]:
dandiset_id = "000336"
dandi_filepath = "sub_1170622509/sub_1170622509sess_1194683535/sub_1170622509+sess_1194683535+exp_1194752582_ophys.nwb"
authenticate = True
dandi_api_key = os.environ["DANDI_API_KEY"]
download_loc = "."

In [4]:
filename = dandi_filepath.split("/")[-1]
filepath = f"{download_loc}/{filename}"

In [5]:
if authenticate:
    client = dandiapi.DandiAPIClient(token=dandi_api_key)
else:
    client = dandiapi.DandiAPIClient()
my_dandiset = client.get_dandiset(dandiset_id)
print(f"Got dandiset {my_dandiset}")

A newer version (0.53.0) of dandi/dandi-cli is available. You are using 0.46.6


Got dandiset DANDI:000336/draft


In [6]:
file = my_dandiset.get_asset_by_path(dandi_filepath)
# this may take awhile, especially if the file to download is large
file.download(filepath)

print(f"Downloaded file to {filepath}")

Downloaded file to ./sub_1170622509+sess_1194683535+exp_1194752582_ophys.nwb


### Extracting 2P Data and Stimulus Data
Below, the fluorescence traces and timestamps are read from the file's `Processing` section. Note that the exact format to access these traces can vary between newer and older NWB files, so some adjustments may be necessary. Additionally, the stimulus data is also read from the NWB file's `Intervals` section. Stimulus information is stored as a series of tables depending on the type of stimulus shown in the session. One such table is displayed below, as well as the names of all the tables.

In [7]:
io = NWBHDF5IO(filepath, mode="r", load_namespaces=True)
nwb = io.read() 

c:\Users\carter.peene\AppData\Local\Programs\Python\Python39\lib\site-packages\hdmf\spec\namespace.py:531: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.5.0 because version 1.5.1 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
c:\Users\carter.peene\AppData\Local\Programs\Python\Python39\lib\site-packages\hdmf\spec\namespace.py:531: UserWarning: Ignoring cached namespace 'core' version 2.3.0 because version 2.6.0-alpha is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
c:\Users\carter.peene\AppData\Local\Programs\Python\Python39\lib\site-packages\hdmf\spec\namespace.py:531: UserWarning: Ignoring cached namespace 'hdmf-experimental' version 0.1.0 because version 0.2.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."


In [8]:
dff = nwb.processing["ophys"]["dff"]
dff_trace = dff.roi_response_series["traces"].data
dff_timestamps = dff.roi_response_series["traces"].timestamps

flr = nwb.processing["ophys"]["corrected_fluorescence"]
flr_trace = flr.roi_response_series["traces"].data
flr_timestamps = flr.roi_response_series["traces"].timestamps

# accessing the above data may look different for older nwb files, like the following
# dff_trace = dff.roi_response_series["RoiResponseSeries"].data
# dff_timestamps = dff.roi_response_series["RoiResponseSeries"].timestamps

print(flr_trace.shape)
print(flr_timestamps.shape)
print(dff_trace.shape)
print(dff_timestamps.shape)

(42337, 103)
(42337,)
(42337, 103)
(42337,)


In [9]:
stimulus_names = list(nwb.intervals.keys())
print(stimulus_names)

['fixed_gabors_presentations', 'gratings_presentations', 'movie_flower_fwd_presentations', 'movie_touch_of_evil_fwd_presentations', 'movie_worms_fwd_presentations', 'rotate_gabors_presentations', 'spontaneous_presentations']


In [10]:
stim_table = nwb.intervals['gratings_presentations']
print(stim_table.colnames)
stim_table[:10]

('start_time', 'stop_time', 'stimulus_name', 'stimulus_block', 'color', 'contrast', 'mask', 'opacity', 'ori', 'size', 'units', 'stimulus_index', 'spatial_frequency', 'phase', 'tags', 'timeseries')


,start_time,stop_time,stimulus_name,stimulus_block,color,contrast,mask,opacity,ori,size,units,stimulus_index,spatial_frequency,phase,tags,timeseries
id,,,,,,,,,,,,,,,,
0,3723.084780,3723.385052,gratings,184.0,"[1.0, 1.0, 1.0]",0.8,None,1.0,60.0,"[250.0, 250.0]",deg,7.0,0.08,0.50,[stimulus_time_interval],"[(222259, 18, timestamps pynwb.base.TimeSeries..."
1,3723.385052,3723.685324,gratings,184.0,"[1.0, 1.0, 1.0]",0.8,None,1.0,90.0,"[250.0, 250.0]",deg,7.0,0.08,0.25,[stimulus_time_interval],"[(222277, 18, timestamps pynwb.base.TimeSeries..."
2,3723.685324,3723.985596,gratings,184.0,"[1.0, 1.0, 1.0]",0.8,None,1.0,150.0,"[250.0, 250.0]",deg,7.0,0.08,0.75,[stimulus_time_interval],"[(222295, 18, timestamps pynwb.base.TimeSeries..."
3,3723.985596,3724.285858,gratings,184.0,"[1.0, 1.0, 1.0]",0.8,None,1.0,120.0,"[250.0, 250.0]",deg,7.0,0.08,0.75,[stimulus_time_interval],"[(222313, 18, timestamps pynwb.base.TimeSeries..."
4,3724.285858,3724.586114,gratings,184.0,"[1.0, 1.0, 1.0]",0.8,None,1.0,0.0,"[250.0, 250.0]",deg,7.0,0.08,0.00,[stimulus_time_interval],"[(222331, 18, timestamps pynwb.base.TimeSeries..."
5,3724.586114,3724.886369,gratings,184.0,"[1.0, 1.0, 1.0]",0.8,None,1.0,30.0,"[250.0, 250.0]",deg,7.0,0.08,0.75,[stimulus_time_interval],"[(222349, 18, timestamps pynwb.base.TimeSeries..."
6,3724.886369,3725.186622,gratings,184.0,"[1.0, 1.0, 1.0]",0.8,None,1.0,150.0,"[250.0, 250.0]",deg,7.0,0.08,0.00,[stimulus_time_interval],"[(222367, 18, timestamps pynwb.base.TimeSeries..."
7,3725.186622,3725.486867,gratings,184.0,"[1.0, 1.0, 1.0]",0.8,None,1.0,120.0,"[250.0, 250.0]",deg,7.0,0.08,0.00,[stimulus_time_interval],"[(222385, 18, timestamps pynwb.base.TimeSeries..."
8,3725.486867,3725.787112,gratings,184.0,"[1.0, 1.0, 1.0]",0.8,None,1.0,60.0,"[250.0, 250.0]",deg,7.0,0.08,0.00,[stimulus_time_interval],"[(222403, 18, timestamps pynwb.base.TimeSeries..."
